In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier , plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn import tree
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.impute import SimpleImputer  

# Read the math placement data from the Excel file
df_math_placement = pd.read_excel('math_placement.xlsx')

# Read the edready raw scores data from the CSV file
df_ed_ready = pd.read_csv('edready_raw_scores.csv')

# Load the file of incoming students
#incoming_students = pd.read_csv('incoming_students.csv')will need to change to correct file name


In [4]:
df_cleaned = df_math_placement.copy()

# Define the mapping of grades to grade categories
grade_mapping = {
    "A": "Successful", "B": "Successful", "B+": "Successful",
    "A-": "Successful", "B*": "Successful", "A*": "Successful",
    "B+*": "Successful", "W": "Unsuccessful", "C-": "Unsuccessful",
    "F": "Unsuccessful", "B-": "Successful", "C+": "Unsuccessful",
    "D": "Unsuccessful", "D*": "Unsuccessful", "C": "Unsuccessful",
    "W*": "Unsuccessful", "P*": "Successful", "C+*": "Unsuccessful",
    "F*": "Unsuccessful", "D+": "Unsuccessful", "P": "Successful",
    "C*": "Unsuccessful", "A-*": "Successful", "I": "Unsuccessful",
    "AU": "Ignore", "B-*": "Successful", "D-": "Unsuccessful",
    "NR": "Ignore", "C-*": "Unsuccessful", "I*": "Unsuccessful",
    "NR*": "Ignore"
}

# Map the grades to their categories and append as a new column
df_cleaned['grade_category'] = df_cleaned['grade'].map(grade_mapping)

import pandas as pd

# Define base courses and adjust the list based on 'campus_code'
base_courses = ['088', '216Q', '132', '161Q', '151Q', '165Q', '171Q']
courses_to_check = ['105Q', '090', '121Q']

for course in courses_to_check:
    if df_cleaned[(df_cleaned['campus_code'] != 'ZGC') & (df_cleaned['course_number'] == course)].shape[0] > 0:
        base_courses.append(course)

# Define course combinations
course_combinations = {
    'Combo1': ('005', '105Q'),
    'Combo2': ('063', '090'),
    'Combo3': ('021', '121Q')
}

# Map courses and combinations to levels
course_levels = {
    '088': 100,
    'Combo2': 100,  # ('063', '090')
    'Combo1': 150,  # ('005', '105Q')
    '090': 150,
    'Combo3': 250,  # ('021', '121Q')
    '105Q': 290,
    '216Q': 300,
    '132': 300,
    '121Q': 300,
    '161Q': 400,
    '151Q': 400,
    '165Q': 500,
    '171Q': 500
}

# Function to determine course level based on the course number or combination
def determine_course_level(course_number, campus_code):
    # Handle special cases for '105Q', '090', '121Q' based on 'campus_code'
    if course_number in ['105Q', '090', '121Q'] and campus_code == 'ZGC':
        # Exclude these courses for ZGC campus
        return None
    for combo_name, combo_courses in course_combinations.items():
        if course_number in combo_courses:
            # Return the level for the course combination
            return course_levels[combo_name]
    # Return the level for individual courses
    return course_levels.get(course_number)

# Apply the function to each row in df_cleaned to create a new 'course_level' column
df_cleaned['course_level'] = df_cleaned.apply(lambda row: determine_course_level(row['course_number'], row['campus_code']), axis=1)

# Create a new column to store the PIDM, course number, and term as a single string
df_cleaned['PIDM_course_number_term'] = df_cleaned['PIDM'].astype(str) + '_' + df_cleaned['course_number'].astype(str) + '_' + df_cleaned['term'].astype(str)

# Remove duplicates based on the new column
df_cleaned = df_cleaned.drop_duplicates(subset='PIDM_course_number_term')


# Make sure the column names match in both dataframes before merging
df_cleaned.rename(columns={'pidm': 'PIDM'}, inplace=True)

# Display the cleaned and imputed dataframe
df_cleaned.head()




,Row,PIDM,term_code,admit_term,degree.x,major_code,college.x,major.x,stu_type,stu_level.x,...,section_number,subj_code,section_type,credit_levl,course_credits,reg_status,grade,grade_category,course_level,PIDM_course_number_term
0,1,41845,202250,202250.0,Master of Science,MATO,College of Letters & Science,Mathematics,N,GR,...,801,M,L,GR,3,RW,A,Successful,NaN,41845_533_202370
1,2,41845,202250,202250.0,Master of Science,MATO,College of Letters & Science,Mathematics,N,GR,...,801,M,L,GR,3,RE,A,Successful,NaN,41845_518_202250
2,3,41845,202250,202250.0,Master of Science,MATO,College of Letters & Science,Mathematics,N,GR,...,1,M,L,GR,3,RE,B,Successful,NaN,41845_524_202350
3,4,41845,202250,202250.0,Master of Science,MATO,College of Letters & Science,Mathematics,N,GR,...,1,M,L,GR,3,RE,A,Successful,NaN,41845_520_202250
4,5,41845,202250,202250.0,Master of Science,MATO,College of Letters & Science,Mathematics,N,GR,...,801,M,L,GR,3,RE,A,Successful,NaN,41845_577_202330


In [5]:

# Convert 'term' to a numeric type if it's not already, to ensure correct sorting
df_cleaned['term'] = pd.to_numeric(df_cleaned['term'])

# Find the oldest term for each PIDM
oldest_terms = df_cleaned.groupby('PIDM')['term'].min().reset_index()

# Merge the oldest terms back to the original DataFrame to filter records
df_oldest = pd.merge(df_cleaned, oldest_terms, on=['PIDM', 'term'], how='inner')

# Now, df_oldest contains only the records of the oldest term for each PIDM,
# including cases where there are multiple records for a PIDM within that oldest term

# Optionally, sort by PIDM and term if needed
df_oldest.sort_values(by=['PIDM', 'term'], ascending=[True, True], inplace=True)

# Show the updated DataFrame
df_oldest.head()


,Row,PIDM,term_code,admit_term,degree.x,major_code,college.x,major.x,stu_type,stu_level.x,...,section_number,subj_code,section_type,credit_levl,course_credits,reg_status,grade,grade_category,course_level,PIDM_course_number_term
0,2,41845,202250,202250.0,Master of Science,MATO,College of Letters & Science,Mathematics,N,GR,...,801,M,L,GR,3,RE,A,Successful,NaN,41845_518_202250
1,4,41845,202250,202250.0,Master of Science,MATO,College of Letters & Science,Mathematics,N,GR,...,1,M,L,GR,3,RE,A,Successful,NaN,41845_520_202250
2,29,149852,202070,202070.0,Associate of Arts,AA,Gallatin College,Associate of Arts,T,UG,...,922,M,L,UG,4,RE,B*,Successful,NaN,149852_090_202270
3,30,149852,202070,202070.0,Associate of Arts,AA,Gallatin College,Associate of Arts,T,UG,...,922,M,L,UG,1,RE,A*,Successful,100.0,149852_063_202270
4,35,210982,202230,202230.0,Bachelor of Science,NURS,College of Nursing,Nursing,T,UG,...,2,STAT,L,UG,3,RW,B,Successful,300.0,210982_216Q_202170


In [6]:
# Trim whitespace from headers
df_oldest.columns = df_oldest.columns.str.strip()
df_ed_ready.columns = df_ed_ready.columns.str.strip()

# Merge the two dataframes based on the PIDM
df_merged = pd.merge(df_oldest, df_ed_ready[['PIDM', 'ERM_SCORE']], on='PIDM', how='left')

# Display the merged dataframe
df_merged.head()


,Row,PIDM,term_code,admit_term,degree.x,major_code,college.x,major.x,stu_type,stu_level.x,...,subj_code,section_type,credit_levl,course_credits,reg_status,grade,grade_category,course_level,PIDM_course_number_term,ERM_SCORE
0,2,41845,202250,202250.0,Master of Science,MATO,College of Letters & Science,Mathematics,N,GR,...,M,L,GR,3,RE,A,Successful,NaN,41845_518_202250,NaN
1,4,41845,202250,202250.0,Master of Science,MATO,College of Letters & Science,Mathematics,N,GR,...,M,L,GR,3,RE,A,Successful,NaN,41845_520_202250,NaN
2,29,149852,202070,202070.0,Associate of Arts,AA,Gallatin College,Associate of Arts,T,UG,...,M,L,UG,4,RE,B*,Successful,NaN,149852_090_202270,10.0
3,30,149852,202070,202070.0,Associate of Arts,AA,Gallatin College,Associate of Arts,T,UG,...,M,L,UG,1,RE,A*,Successful,100.0,149852_063_202270,10.0
4,35,210982,202230,202230.0,Bachelor of Science,NURS,College of Nursing,Nursing,T,UG,...,STAT,L,UG,3,RW,B,Successful,300.0,210982_216Q_202170,NaN


In [7]:
# Count the number of rows in the merged dataframe
num_rows = df_merged.shape[0]
print(f"Number of rows in df_merged: {num_rows}")


Number of rows in df_merged: 17180


In [8]:

# Count the occurrences of each grade category
grade_counts = df_merged['grade_category'].value_counts()

# Display the count of each grade category
print(grade_counts)


grade_category
Successful      10124
Unsuccessful     6977
Ignore             65
Name: count, dtype: int64


## Model 1: Logistic Regression Model - ERM Score as Predictor Variable & Grade Category as Target Variable

In [9]:
# Drop NaN values from the 'hs_gpa' and 'ERM_SCORE' columns
df_merged.dropna(subset=[ 'ERM_SCORE'], inplace=True)

# Define base courses and course combinations
base_courses = ['088', '216Q', '132', '161Q', '151Q', '165Q', '171Q']
course_combinations = {
    'Combo1': ('005', '105Q'),
    'Combo2': ('063', '090'),
    'Combo3': ('021', '121Q')
}

# Add '105Q', '090', '121Q' conditionally based on campus_code
courses_to_check = ['105Q', '090', '121Q']
for course in courses_to_check:
    if df_merged[(df_merged['campus_code'] != 'ZGC') & (df_merged['course_number'] == course)].shape[0] > 0:
        base_courses.append(course)

# Combine individual courses with combination labels for iteration
included_courses = base_courses + list(course_combinations.keys())

# Filtering DataFrame based on updated logic
def filter_df(df, course_key):
    # For combinations, select rows matching any of the combination courses
    if course_key in course_combinations:
        return df[df['course_number'].isin(course_combinations[course_key])]
    # For individual courses, simply filter by the course number
    else:
        return df[df['course_number'] == course_key]

# Initialize model storage
models = {}

for course_key in included_courses:
    df_filtered = filter_df(df_merged, course_key)
    if df_filtered.empty:
        print(f"No data for {course_key}")
        continue

    X = df_filtered[['ERM_SCORE']]
    y = df_filtered['grade_category']

    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)

    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.20, random_state=42)

    log_reg = LogisticRegression(max_iter=1000)
    log_reg.fit(X_train, y_train)

    y_pred = log_reg.predict(X_test)

    # Handling mismatch in class sizes for the classification report
    unique_y_test = set(y_test)
    unique_y_pred = set(y_pred)
    unique_classes = unique_y_test.union(unique_y_pred)
    target_names = label_encoder.inverse_transform(list(unique_classes))

    print(f'Course/Combo: {course_key}')
    print(f'Accuracy: {accuracy_score(y_test, y_pred):.2f}')
    print(classification_report(y_test, y_pred, labels=list(unique_classes), target_names=target_names, zero_division=1))
    print("--------------------\n")

    # Store the trained model
    models[course_key] = log_reg



Course/Combo: 088
Accuracy: 0.53
              precision    recall  f1-score   support

  Successful       1.00      0.00      0.00         7
Unsuccessful       0.53      1.00      0.70         8

    accuracy                           0.53        15
   macro avg       0.77      0.50      0.35        15
weighted avg       0.75      0.53      0.37        15

--------------------

Course/Combo: 216Q
Accuracy: 0.48
              precision    recall  f1-score   support

  Successful       0.43      0.58      0.49        36
Unsuccessful       0.56      0.40      0.47        47

    accuracy                           0.48        83
   macro avg       0.49      0.49      0.48        83
weighted avg       0.50      0.48      0.48        83

--------------------

Course/Combo: 132
Accuracy: 0.60
              precision    recall  f1-score   support

  Successful       0.60      1.00      0.75         9
Unsuccessful       1.00      0.00      0.00         6

    accuracy                          

## Model 2: Logistic Regression Model - ERM Score & High School GPA as Predictor Variables & Grade Category as Target Variable

In [ ]:
# Drop NaN values from the 'hs_gpa' and 'ERM_SCORE' columns
df_merged.dropna(subset=[ 'ERM_SCORE','hs_gpa'], inplace=True)

# Define base courses and course combinations
base_courses = ['088', '216Q', '132', '161Q', '151Q', '165Q', '171Q']
course_combinations = {
    'Combo1': ('005', '105Q'),
    'Combo2': ('063', '090'),
    'Combo3': ('021', '121Q')
}

# Add '105Q', '090', '121Q' conditionally based on campus_code
courses_to_check = ['105Q', '090', '121Q']
for course in courses_to_check:
    if df_merged[(df_merged['campus_code'] != 'ZGC') & (df_merged['course_number'] == course)].shape[0] > 0:
        base_courses.append(course)

# Combine individual courses with combination labels for iteration
included_courses = base_courses + list(course_combinations.keys())

# Filtering DataFrame based on updated logic
def filter_df(df, course_key):
    # For combinations, select rows matching any of the combination courses
    if course_key in course_combinations:
        return df[df['course_number'].isin(course_combinations[course_key])]
    # For individual courses, simply filter by the course number
    else:
        return df[df['course_number'] == course_key]

# Initialize model storage
models = {}

for course_key in included_courses:
    df_filtered = filter_df(df_merged, course_key)
    if df_filtered.empty:
        print(f"No data for {course_key}")
        continue

    X = df_filtered[['ERM_SCORE','hs_gpa']]
    y = df_filtered['grade_category']

    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)

    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.20, random_state=42)

    log_reg = LogisticRegression(max_iter=1000)
    log_reg.fit(X_train, y_train)

    y_pred = log_reg.predict(X_test)

    # Handling mismatch in class sizes for the classification report
    unique_y_test = set(y_test)
    unique_y_pred = set(y_pred)
    unique_classes = unique_y_test.union(unique_y_pred)
    target_names = label_encoder.inverse_transform(list(unique_classes))

    print(f'Course/Combo: {course_key}')
    print(f'Accuracy: {accuracy_score(y_test, y_pred):.2f}')
    print(classification_report(y_test, y_pred, labels=list(unique_classes), target_names=target_names, zero_division=1))
    print("--------------------\n")

    # Store the trained model
    models[course_key] = log_reg



In [10]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression

# Assuming df_merged is your initial DataFrame
# Ensure it's loaded or defined before this point

# Removing rows with NA in 'hs_gpa' or 'ERM_SCORE' columns before any processing
df_merged.dropna(subset=['hs_gpa', 'ERM_SCORE'], inplace=True)

# Prepare the incoming_students DataFrame by explicitly creating it or filtering df_merged
incoming_students = df_merged.copy()

# Define the list of courses, including combinations as tuples
included_courses = [
    '088', '105Q', '090', '216Q', '132', '121Q', '161Q', '151Q', '165Q', '171Q',
    ('005', '105Q'), ('063', '090'), ('021', '121Q')
]

# Initialize a dictionary to store each course's LabelEncoder and model
model_info = {}

for course in included_courses:
    # Filter the DataFrame for the current course or combination
    if isinstance(course, tuple):
        df_filtered = df_merged[df_merged['course_number'].isin(course)]
    else:
        df_filtered = df_merged[df_merged['course_number'] == course]

    # Continue only if there are records after filtering
    if not df_filtered.empty:
        X = df_filtered[['hs_gpa', 'ERM_SCORE']]
        y = df_filtered['grade_category']

        # Encoding the target variable for the current course
        label_encoder = LabelEncoder()
        y_encoded = label_encoder.fit_transform(y)

        # Train the logistic regression model
        log_reg = LogisticRegression(max_iter=1000)
        log_reg.fit(X, y_encoded)

        # Store the trained model and its label encoder in the dictionary
        model_info[course] = {'model': log_reg, 'encoder': label_encoder}

# Placeholder DataFrame for storing predictions
predictions = pd.DataFrame(index=incoming_students.index)

# Iterating over the models to predict outcomes for incoming students
for course, info in model_info.items():
    model = info['model']
    encoder = info['encoder']
    
    # Predicting with the model
    pred_encoded = model.predict(incoming_students[['hs_gpa', 'ERM_SCORE']])

    # Inversely transforming encoded predictions into original labels
    pred_decoded = encoder.inverse_transform(pred_encoded)

    # Storing predictions in the DataFrame under the course name or combination
    course_key = '&'.join(course) if isinstance(course, tuple) else course
    predictions[course_key] = pred_decoded

# Defining a function to determine the highest-level course for which a student is predicted to be successful
def determine_highest_level(row):
    # Example logic for determining the highest-level course
    # You may need to adapt this based on your course hierarchy and success criteria
    for course in reversed(included_courses):
        course_key = '&'.join(course) if isinstance(course, tuple) else course
        if row.get(course_key) == 'Successful':
            return course_key
    return 'None'

# Apply the function to each row in predictions DataFrame
predictions['Highest_Level_Course'] = predictions.apply(determine_highest_level, axis=1)

output_file = 'Predictions.csv'

# Join df_merged with predictions using an appropriate method, e.g., merge, join, concat
# Ensure the join keys or indexes align properly between df_merged and predictions
# For simplicity, I'll assume df_merged and predictions can be directly concatenated
# You may need to adjust based on your actual DataFrame structures
df_merged_with_predictions = pd.concat([df_merged, predictions], axis=1)

# Save the merged dataframe to a CSV file
df_merged_with_predictions.to_csv(output_file, index=False)

print(f"Predictions saved to '{output_file}'.")


Predictions saved to 'Predictions.csv'.


In [ ]:

# Drop NaN values from 'hs_gpa' and 'ERM_SCORE' columns
df_merged.dropna(subset=['hs_gpa', 'ERM_SCORE'], inplace=True)

# Define base courses and course combinations
base_courses = ['088', '216Q', '132', '161Q', '151Q', '165Q', '171Q']
course_combinations = {
    'Combo1': ('005', '105Q'),
    'Combo2': ('063', '090'),
    'Combo3': ('021', '121Q')
}

# Add '105Q', '090', '121Q' conditionally based on campus_code
courses_to_check = ['105Q', '090', '121Q']
for course in courses_to_check:
    if df_merged[(df_merged['campus_code'] != 'ZGC') & (df_merged['course_number'] == course)].shape[0] > 0:
        base_courses.append(course)

# Combine individual courses with combination labels for iteration
included_courses = base_courses + list(course_combinations.keys())

# Filtering DataFrame based on updated logic
def filter_df(df, course_key):
    # For combinations, select rows matching any of the combination courses
    if course_key in course_combinations:
        return df[df['course_number'].isin(course_combinations[course_key])]
    # For individual courses, simply filter by the course number
    else:
        return df[df['course_number'] == course_key]

# Now, `included_courses` correctly reflects the campus_code restriction for '105Q', '090', '121Q',
# while `course_combinations` are processed to include all campus codes, including 'ZGC'.

# Initialize model storage
models = {}

for course_key in included_courses:
    # Use the filter_df function to filter the DataFrame
    df_filtered = filter_df(df_merged, course_key)

    if not df_filtered.empty:
        X = df_filtered[['hs_gpa', 'ERM_SCORE']]
        y = df_filtered['grade_category']

        label_encoder = LabelEncoder()
        y_encoded = label_encoder.fit_transform(y)

        X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.20, random_state=42)

        log_reg = LogisticRegression(max_iter=1000)
        log_reg.fit(X_train, y_train)

        y_pred = log_reg.predict(X_test)

        # Handling mismatch in class sizes for the classification report
        unique_y_test = set(y_test)
        unique_y_pred = set(y_pred)
        unique_classes = unique_y_test.union(unique_y_pred)
        target_names = label_encoder.inverse_transform(list(unique_classes))

        print(f'Course/Combo: {course_key}')
        print(f'Accuracy: {accuracy_score(y_test, y_pred):.2f}')
        print(classification_report(y_test, y_pred, labels=list(unique_classes), target_names=target_names, zero_division=0))
        print("--------------------\n")

        # Store the trained model
        models[course_key] = log_reg


## Model 3: Random Forest

In [ ]:
# Drop rows with NaN values in either 'hs_gpa' or 'ERM_SCORE' columns
df_filtered_clean = df_filtered.dropna(subset=['hs_gpa', 'ERM_SCORE'])

# Now prepare your predictors (X) and target (y) using the cleaned DataFrame
X = df_filtered_clean[['hs_gpa', 'ERM_SCORE']]
y = df_filtered_clean['grade_category']

# Training and testing split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a decision tree classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Predict the target
y_pred = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print(classification_report(y_test, y_pred))

## Model 4:  Decision Tree

In [ ]:
# Making a copy of the dataframe to avoid SettingWithCopyWarning
df_merged_cleaned = df_filtered_clean.copy()

# Ensure no NaN values in predictors
X = X.dropna()

# Encode the target variable 'course_number'
course_number_encoder = LabelEncoder()
df_merged_cleaned['course_number_encoded'] = course_number_encoder.fit_transform(df_merged_cleaned['course_number'])

# Predictors
X = df_merged_cleaned[['hs_gpa', 'ERM_SCORE']]

# Target variable is'course_number_encoded'
y_course_number = df_merged_cleaned['course_number_encoded']

# Split the data into training and testing sets
X_train, X_test, y_train_course_number, y_test_course_number = train_test_split(X, y_course_number, test_size=0.3, random_state=42)

# Initialize the DecisionTreeClassifier with parameters to control tree complexity
clf_course_number = DecisionTreeClassifier(
    max_depth=5,               # Limit the depth of the tree
    min_samples_split=40,      # Require at least 40 samples to split a node
    min_samples_leaf=20,       # Each leaf node must contain at least 20 samples
    max_leaf_nodes=15,         # Limit the total number of leaf nodes
    random_state=42
)
# Fit the classifier to the training data
clf_course_number.fit(X_train, y_train_course_number)

# Predict on the test data
y_pred_course_number = clf_course_number.predict(X_test)


## Model 5: XG Boost

In [ ]:
# Decode the predicted course numbers back to the original encoding
predicted_courses = course_number_encoder.inverse_transform(y_pred_course_number)

# Add the predicted courses to your test DataFrame
X_test.loc[:, 'predicted_course_number'] = predicted_courses

# Join the original 'grade_category' to the test DataFrame for evaluation
X_test = X_test.join(df_merged_cleaned['grade_category'], how='left')

# Evaluate the success by checking the grade_category of the predicted placements
successful_placements = X_test[X_test['grade_category'] == 'Successful']
success_rate = len(successful_placements) / len(X_test)
print(f'Success rate of placements: {success_rate:.2f}')


In [ ]:
#Clean the data
df_filtered_clean = df_filtered.dropna(subset=['ERM_SCORE', 'hs_gpa'])


#Filter out classes with only one instance
class_counts = df_merged_cleaned['course_number'].value_counts()
df_filtered = df_merged_cleaned[df_merged_cleaned['course_number'].isin(class_counts[class_counts > 1].index)].copy()

#Encode the target variable
course_number_encoder = LabelEncoder()
df_filtered['course_number_encoded'] = course_number_encoder.fit_transform(df_filtered['course_number'])

#Prepare the data
X = df_filtered_clean[['ERM_SCORE', 'hs_gpa']]
y = df_filtered_clean['course_number_encoded'] 

# Ensure no NaN values in predictors
X = X.dropna()

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Continue with initializing and fitting your XGBoost classifier
xgb_model_course_number = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
xgb_model_course_number.fit(X_train, y_train)

# Predict on the test data
y_pred_course_number = xgb_model_course_number.predict(X_test)

# Decode the predicted course numbers back to the original course numbers for interpretability
predicted_course_numbers = course_number_encoder.inverse_transform(y_pred_course_number)

# Evaluate the predictions
accuracy = accuracy_score(y_test, y_pred_course_number)  # This comparison should now be valid
print(f'Accuracy of the XGBoost model for course number prediction: {accuracy:.2f}')
